In [135]:
import ast
import json
from tqdm.auto import tqdm
import pandas as pd 
import re 
import sys
sys.path.insert(0, '../tasks/generate_synthetic_example_data/')
import run_synthetic_data_models as r
import model_util as m_utils
import warnings
warnings.simplefilter(action='ignore', category=SyntaxWarning)

tok, gpt_4_model,_ = r.get_model('gpt-4-turbo', cache_dir=None)

In [93]:
from importlib import reload
reload(m)

<module 'model_util' from '/Users/spangher/Projects/usc-research/style-edits/notebooks/../tasks/generate_synthetic_example_data/model_util.py'>

In [2]:
proj_dir = '/project/jonmay_231/spangher/Projects/style-guides'
! ssh end "ls -lh $proj_dir/tasks/generate_synthetic_example_data/ | grep bad_only.jsonl"

-rw-rw---- 1 spangher jonmay_231  984K Apr 23 23:24 gpt-3.5-turbo_zeroshot_bad_only.jsonl
-rw-rw---- 1 spangher jonmay_231  508K Apr 23 23:47 gpt-4-turbo_zeroshot_bad_only.jsonl
-rw-rw---- 1 spangher jonmay_231     0 Apr 23 22:31 llama-3-70b_zeroshot_bad_only.jsonl
-rw-rw---- 1 spangher jonmay_231     0 Apr 23 22:31 mixtral_zeroshot_bad_only.jsonl


In [ ]:
! scp end:$proj_dir/tasks/generate_synthetic_example_data/gpt-4-turbo_zeroshot_bad_only.jsonl ../tasks/generate_synthetic_example_data/
! scp end:$proj_dir/tasks/generate_synthetic_example_data/gpt-3.5-turbo_zeroshot_bad_only.jsonl ../tasks/generate_synthetic_example_data/
# ! scp end:$proj_dir/tasks/generate_synthetic_example_data/command-r_zeroshot.jsonl ../tasks/generate_synthetic_example_data/
# ! scp end:$proj_dir/tasks/generate_synthetic_example_data/mixtral_zeroshot.jsonl ../tasks/generate_synthetic_example_data/

In [18]:
# mixtral_data_df = pd.read_json('../tasks/generate_synthetic_example_data/mixtral_zeroshot.jsonl', lines=True)
# command_r_data_df = pd.read_json('../tasks/generate_synthetic_example_data/command-r_zeroshot.jsonl', lines=True)
input_data_df = pd.read_json('../corpora/chicago-style-guide/all-rules.jsonl', lines=True)
gpt4_data_df = pd.read_json('../tasks/generate_synthetic_example_data/gpt-4-turbo_zeroshot_bad_only.jsonl', lines=True)
gpt3_data_df = pd.read_json('../tasks/generate_synthetic_example_data/gpt-3.5-turbo_zeroshot_bad_only.jsonl', lines=True)

In [38]:
from run_json_parsing_models import parse_data, WAS_UNCLEAR_STR

In [31]:
parsed_data, failed = gpt4_data_df.pipe(parse_data, return_failed=True,  verbose=True)

  0%|          | 0/521 [00:00<?, ?it/s]

In [36]:
gpt4_data_df['parsed_output'] = parsed_data

In [43]:
input_data_df_w_all_annotations = (
    input_data_df
    ## gpt 4
         .merge(gpt4_data_df.drop(columns=['message']).rename(columns={'key': 'url'}))
         .assign(was_unclear_gpt4=lambda df: df['parsed_output'] == WAS_UNCLEAR_STR)
         .assign(parsed_output=lambda df: df['parsed_output'].apply(lambda x: x if x != WAS_UNCLEAR_STR else None))
         .rename(columns={'parsed_output': 'parsed_gpt4'})
    # ##  mixtral
    #      .merge(mixtral_data_df.drop(columns=['message']).rename(columns={'key': 'url'}), how='left')
    #      .assign(was_unclear_mixtral=lambda df: df['parsed_output'] == WAS_UNCLEAR_STR)
    #      .assign(parsed_output=lambda df: df['parsed_output'].apply(lambda x: x if x != WAS_UNCLEAR_STR else None))
    #      .rename(columns={'parsed_output': 'parsed_mixtral'})
    # ##  command-r
    #      .merge(command_r_data_df.drop(columns=['message']).rename(columns={'key': 'url'}), how='left')
    #      .assign(was_unclear_command_r=lambda df: df['parsed_output'] == WAS_UNCLEAR_STR)
    #      .assign(parsed_output=lambda df: df['parsed_output'].apply(lambda x: x if x != WAS_UNCLEAR_STR else None))
    #      .rename(columns={'parsed_output': 'parsed_command_r'})
)

In [278]:
from unidecode import unidecode
import re 

print(input_data_df_w_all_annotations
     [['content', 'parsed_gpt4']]
     .iloc[400]
     .pipe(lambda s: print(re.sub('\n+', '\n', s['content']) + '\n\nBad examples:\n' + json.dumps(s['parsed_gpt4'], indent=4, )))
)

Names of persons and other capitalized nouns normally form the plural by adding s or es. Exceptions, including the last example, are generally listed in Webster’s (see also 7.10).
Tom, Dick, and Harry; pl. Toms, Dicks, and Harrys
the Jones family, pl. the Joneses
the Martinez family, pl. the Martinezes
the Bruno family, pl. the Brunos
Sunday, pl. Sundays
Germany, pl. Germanys
Pakistani, pl. Pakistanis
but
Romany, pl. Romanies
An apostrophe is never used to form the plural of a family name: “The Jeffersons live here” (not “Jefferson’s”). For the apostrophe in the possessive form of proper nouns, see 7.17.

Bad examples:
[
    "The Smith's are hosting a community gathering this weekend to celebrate the end of summer.",
    "Several new McDonald's have opened across the city, offering exciting new menu options.",
    "The Johnson's have been recognized for their philanthropic efforts at the annual charity ball.",
    "A group of Picasso's were recently discovered in an abandoned warehouse

In [106]:
all_synthetic_data = (
    input_data_df_w_all_annotations
        [['url', 'title', 'content', 'parsed_gpt4']]#, 'parsed_mixtral', 'parsed_command_r']]
        .set_index(['url', 'title', 'content'])
        .stack()
        .reset_index()
        .rename(columns={'level_3': 'algorithm', 0: 'synthetic_data'})
        .explode('synthetic_data')
        .reset_index(drop=True)
        .reset_index().rename(columns={'index': 'datum_idx'})
)

grouped_synthetic_data = (
    all_synthetic_data
        .groupby(['url', 'title', 'algorithm', 'content'])
        [['datum_idx', 'synthetic_data']]
        .aggregate(list)
)

In [107]:
all_synthetic_data

,datum_idx,url,title,content,algorithm,synthetic_data
0,0,/book/ed17/part2/ch05/psec001.html,The field of grammar,"In its usual sense, grammar is the set of rule...",parsed_gpt4,"Many people in the city, they are moving to su..."
1,1,/book/ed17/part2/ch05/psec001.html,The field of grammar,"In its usual sense, grammar is the set of rule...",parsed_gpt4,A new law have been passed that bans smoking i...
2,2,/book/ed17/part2/ch05/psec001.html,The field of grammar,"In its usual sense, grammar is the set of rule...",parsed_gpt4,The Prime Minister and their cabinet is meetin...
3,3,/book/ed17/part2/ch05/psec001.html,The field of grammar,"In its usual sense, grammar is the set of rule...",parsed_gpt4,There's many reasons why the company's profits...
4,4,/book/ed17/part2/ch05/psec001.html,The field of grammar,"In its usual sense, grammar is the set of rule...",parsed_gpt4,The team of researchers has find a new method ...
...,...,...,...,...,...,...
5199,5199,/book/ed17/part2/ch08/psec040.html,Class,Terms denoting socioeconomic classes or groups...,parsed_gpt4,Recent protests highlight the struggles of The...
5200,5200,/book/ed17/part2/ch08/psec040.html,Class,Terms denoting socioeconomic classes or groups...,parsed_gpt4,The 1 Percent are often blamed for economic di...
5201,5201,/book/ed17/part2/ch08/psec040.html,Class,Terms denoting socioeconomic classes or groups...,parsed_gpt4,An Upper-Middle-Class Family struggles with th...
5202,5202,/book/ed17/part2/ch08/psec040.html,Class,Terms denoting socioeconomic classes or groups...,parsed_gpt4,Middle-Class Neighborhoods are seeing a rise i...


In [110]:
import prompts as p
reload(p)

<module 'prompts' from '/Users/spangher/Projects/usc-research/style-edits/notebooks/../tasks/generate_synthetic_example_data/prompts.py'>

In [175]:
k = 10
all_messages = []
for (url, title, alg, rule),  examples in (
    grouped_synthetic_data
        .apply(lambda x: list(zip(x['datum_idx'], x['synthetic_data'])), axis=1)
        .items()
):
    for batch in m_utils.batchify(examples, n=k):
        batch_idx, batch_text = list(zip(*batch))
        batch = '\n'.join(list(map(lambda x: f"{x[0]+1}. {x[1]}", enumerate(batch_text))))
        messages = {
            'url': url,
            'algorithm': alg,
            'batch_text': list(batch_text),
            'batch_idx': list(batch_idx),
            'messages': [
                {
                    "role": "system",
                    "content": "You are a helpful editor who checks other editors."
                }, {
                    'role': 'user',
                    'content': p.ZERO_SHOT_BATCH_BAD_SENTENCE_SEMANTIC_CHECK.format(
                        rule_title=title, rule=rule, k=k, sentences=batch
                    )
                }
            ]
        }
        all_messages.append(messages)

In [ ]:
all_responses = []
for m in tqdm(all_messages):
    response = gpt_4_model(messages=m['messages'] )
    all_responses.append(response.choices[0].message.content)

In [183]:
all_responses[3]

'1. yes\n2. yes\n3. no\n4. no\n5. no\n6. no\n7. no\n8. no\n9. no\n10. no'

In [190]:
all_messages_df = (
    pd.DataFrame(all_messages)
        .merge(pd.DataFrame(all_responses, columns=['responses']), left_index=True, right_index=True)
)

In [201]:
all_messages_df['responses'] = (
    all_messages_df['responses']
     .str.replace('\d+\. ', '', regex=True).str.split('\n')
)

In [213]:
all_synthetic_data_w_responses = (
    all_synthetic_data
        .merge(
            right=all_messages_df[['batch_idx', 'responses']].explode(['batch_idx', 'responses']),
            left_on='datum_idx',
            right_on='batch_idx'
        )
).assign(responses=lambda df: df['responses'].str.lower())

In [216]:
all_synthetic_data_w_responses['responses'].value_counts()

responses
yes    331
no     219
Name: count, dtype: int64

In [264]:
(all_synthetic_data_w_responses
     .groupby(['url', 'title', 'content'])['responses']
     .value_counts().unstack().fillna(0)
     .apply(lambda x: x['yes'] / (x['yes'] + x['no']), axis=1)
     .sort_values()
     .reset_index()#[0]#.hist(bins=10)
)

,url,title,content,0
0,/book/ed17/part2/ch05/psec018.html,"Common case, nominative function",The nominative (sometimes called the subjectiv...,0.0
1,/book/ed17/part2/ch05/psec026.html,Adverbial functions,Words that are ordinarily nouns occasionally f...,0.0
2,/book/ed17/part2/ch05/psec013.html,Plurals generally,"Because exceptions abound, a good dictionary o...",0.0
3,/book/ed17/part2/ch05/psec028.html,Antecedents of pronouns,A pronoun typically refers to an antecedent—th...,0.0
4,/book/ed17/part2/ch05/psec009.html,Noun case,"In English, only nouns and pronouns have case....",0.0
5,/book/ed17/part2/ch05/psec008.html,Properties of nouns,Nouns have properties of case and number. Some...,0.0
6,/book/ed17/part2/ch05/psec012.html,Noun person,A few grammarians attribute the property of pe...,0.0
7,/book/ed17/part2/ch05/psec030.html,Pronouns without antecedents,Some pronouns do not require antecedents. The ...,0.0
8,/book/ed17/part2/ch05/psec005.html,Common nouns,A common noun is the generic name of one item ...,0.2
9,/book/ed17/part2/ch05/psec047.html,Special uses of personal pronouns,Some personal pronouns have special uses. (1) ...,0.3


In [265]:
t= 'Case denotes the relationship between a noun or pronoun and other words in a sentence.'
t= 'Number shows whether one object or more than one object is referred to, as with clock (singular) and clocks (plural).'
t= 'Nouns have properties of case and number. Some traditional grammarians also consider gender and person to be properties of nouns.'
u='/book/ed17/part2/ch05/psec004.html'
u='/book/ed17/part2/ch05/psec034.html'

In [273]:
(all_synthetic_data_w_responses
     .loc[lambda df: df['url'] == u]
     [['title', 'content', 'synthetic_data', 'responses']]
     .iloc[5]
     .to_dict()
)

{'title': 'Pronoun with multiple antecedents',
 'content': 'When a pronoun has two or more antecedents that differ from the pronoun in person, and the antecedents are connected by and, or, or nor, the pronoun must take the person of only one antecedent. The first person is preferred to the second, and the second person to the third. For example, if the antecedents are in the second and first person, the pronoun that follows is in the first person {you or I should get to work on our experiment [our is in the first person, as is the antecedent I]}. If the antecedents are in the second and third person, the pronoun that follows takes the second person {you and she can settle your dispute}. If the pronoun refers to only one of the connected nouns or pronouns, it takes the person of that noun {you and Marian have discussed her trip report}. At times the pronoun may refer to an antecedent that is not expressed in the same sentence; it takes the number of that antecedent, not of any connected